In [39]:
import pandas as pd
import numpy as np
import joblib
import zipfile
import os
import matplotlib as plt

### Loading data

In [40]:
# Load the model
consumption = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/train.csv') # Replace with "pickle.load" if you used pickle
building = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/building_metadata.csv')
weather = pd.read_csv('/Users/goksuuzunturk/Desktop/DI 502 Project/OriginalDataset/weather_train.csv')


In [41]:
len(consumption[consumption['meter_reading']<= 0])/len(consumption[consumption['meter'] == 0])

0.1553760039665332

In [42]:
consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


In [43]:
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   int64  
 1   building_id  1449 non-null   int64  
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   int64  
 4   year_built   675 non-null    float64
 5   floor_count  355 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.1+ KB


In [44]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int64  
 1   timestamp           139773 non-null  object 
 2   air_temperature     139718 non-null  float64
 3   cloud_coverage      70600 non-null   float64
 4   dew_temperature     139660 non-null  float64
 5   precip_depth_1_hr   89484 non-null   float64
 6   sea_level_pressure  129155 non-null  float64
 7   wind_direction      133505 non-null  float64
 8   wind_speed          139469 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


### Filtering consumption only for electrictiy readings

In [45]:
electricity = consumption[(consumption['meter'] == 0)]

In [46]:
electricity = electricity.drop(columns='meter')

In [47]:
electricity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12060910 entries, 0 to 20216099
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   timestamp      object 
 2   meter_reading  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 368.1+ MB


In [48]:
electricity.describe()

,building_id,meter_reading
count,1.206091e+07,1.206091e+07
mean,7.066480e+02,1.708256e+02
std,4.152330e+02,3.808343e+02
min,0.000000e+00,0.000000e+00
25%,3.450000e+02,1.950000e+01
50%,6.980000e+02,6.283190e+01
75%,1.063000e+03,1.700750e+02
max,1.448000e+03,7.976900e+04


In [49]:
electricity.describe()

,building_id,meter_reading
count,1.206091e+07,1.206091e+07
mean,7.066480e+02,1.708256e+02
std,4.152330e+02,3.808343e+02
min,0.000000e+00,0.000000e+00
25%,3.450000e+02,1.950000e+01
50%,6.980000e+02,6.283190e+01
75%,1.063000e+03,1.700750e+02
max,1.448000e+03,7.976900e+04


### Merging everything into two datasets

In [50]:
electricity_building = electricity.merge(building, on='building_id', how='left')


electricity_building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   timestamp      object 
 2   meter_reading  float64
 3   site_id        int64  
 4   primary_use    object 
 5   square_feet    int64  
 6   year_built     float64
 7   floor_count    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 736.1+ MB


In [51]:
electricity_building_weather = electricity_building.merge(weather, on=['site_id', 'timestamp'], how='left')


electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12060910 entries, 0 to 12060909
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int64  
 1   timestamp           object 
 2   meter_reading       float64
 3   site_id             int64  
 4   primary_use         object 
 5   square_feet         int64  
 6   year_built          float64
 7   floor_count         float64
 8   air_temperature     float64
 9   cloud_coverage      float64
 10  dew_temperature     float64
 11  precip_depth_1_hr   float64
 12  sea_level_pressure  float64
 13  wind_direction      float64
 14  wind_speed          float64
dtypes: float64(10), int64(3), object(2)
memory usage: 1.3+ GB


### Drop buildings where year_built >= 2016

In [52]:
electricity_building_weather = electricity_building_weather[(electricity_building_weather['year_built'] < 2016) | (electricity_building_weather['year_built'].isna()) ]

### Filter hours for 8.00-18.00

In [53]:
electricity_building_weather = electricity_building_weather[(pd.to_datetime(electricity_building_weather['timestamp']).dt.hour >= 8) & (pd.to_datetime(electricity_building_weather['timestamp']).dt.hour <= 18)]

### Drop weekends

In [54]:
electricity_building_weather = electricity_building_weather[pd.to_datetime(electricity_building_weather['timestamp']).dt.dayofweek <= 4]

In [55]:
electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3920068 entries, 11096 to 12020212
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int64  
 1   timestamp           object 
 2   meter_reading       float64
 3   site_id             int64  
 4   primary_use         object 
 5   square_feet         int64  
 6   year_built          float64
 7   floor_count         float64
 8   air_temperature     float64
 9   cloud_coverage      float64
 10  dew_temperature     float64
 11  precip_depth_1_hr   float64
 12  sea_level_pressure  float64
 13  wind_direction      float64
 14  wind_speed          float64
dtypes: float64(10), int64(3), object(2)
memory usage: 478.5+ MB


### Site_id = 0 Measurement Correction

In [56]:
electricity_building_weather[electricity_building_weather['site_id']==0].meter_reading.describe()

count    282578.000000
mean        256.835529
std         416.634834
min           0.000000
25%           0.000000
50%          88.050300
75%         362.781000
max        2848.320000
Name: meter_reading, dtype: float64

In [57]:
electricity_building_weather.loc[electricity_building_weather['site_id']==0,'meter_reading'] *= 0.293014534

In [58]:
electricity_building_weather[electricity_building_weather['site_id']==0].meter_reading.describe()

count    282578.000000
mean         75.256543
std         122.080062
min           0.000000
25%           0.000000
50%          25.800018
75%         106.300106
max         834.599157
Name: meter_reading, dtype: float64

In [59]:
electricity_building_weather

,building_id,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
11096,0,2016-01-01 08:00:00,0.000,0,Education,7432,2008.0,NaN,20.6,NaN,20.0,0.0,1018.1,0.0,0.0
11097,1,2016-01-01 08:00:00,0.000,0,Education,2720,2004.0,NaN,20.6,NaN,20.0,0.0,1018.1,0.0,0.0
11098,2,2016-01-01 08:00:00,0.000,0,Education,5376,1991.0,NaN,20.6,NaN,20.0,0.0,1018.1,0.0,0.0
11099,3,2016-01-01 08:00:00,0.000,0,Education,23685,2002.0,NaN,20.6,NaN,20.0,0.0,1018.1,0.0,0.0
11100,4,2016-01-01 08:00:00,0.000,0,Education,116607,1975.0,NaN,20.6,NaN,20.0,0.0,1018.1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12020208,1444,2016-12-30 18:00:00,8.625,15,Entertainment/public assembly,19619,1914.0,NaN,-1.7,NaN,-6.1,-1.0,1004.7,290.0,11.3
12020209,1445,2016-12-30 18:00:00,5.775,15,Education,4298,NaN,NaN,-1.7,NaN,-6.1,-1.0,1004.7,290.0,11.3
12020210,1446,2016-12-30 18:00:00,0.000,15,Entertainment/public assembly,11265,1997.0,NaN,-1.7,NaN,-6.1,-1.0,1004.7,290.0,11.3
12020211,1447,2016-12-30 18:00:00,166.775,15,Lodging/residential,29775,2001.0,NaN,-1.7,NaN,-6.1,-1.0,1004.7,290.0,11.3


### Analyze data

In [60]:
electricity_building_weather[electricity_building_weather['meter_reading']==0].site_id.unique()

array([ 0,  3,  4,  5,  7,  8,  9, 13, 15, 12,  2, 14, 10,  6])

In [61]:
electricity_building_weather.groupby('site_id').timestamp.count()

site_id
0     282578
1     146419
2     387102
3     768646
4     242198
5     255519
6     102684
7      34248
8     186506
9     349462
10     77389
11     14208
12    103265
13    403973
14    291092
15    274779
Name: timestamp, dtype: int64

### Saving filtered dataset

In [62]:
# Specify the zip file name
zip_filename = "../dataset/filtered.zip"

# Create a ZIP file and add the model object to it
with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as archive:
    # Save the model to a temporary file
    temp_model_filename = "filtered.pkl"
    joblib.dump(electricity_building_weather, temp_model_filename)
    
    # Add the temporary model file to the ZIP archive
    archive.write(temp_model_filename, arcname="filtered.pkl")

# Remove the temporary model file
os.remove(temp_model_filename)
